<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Dylan/Seeded_CNMF_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install CaImAn - takes around 2 minutes

!git clone https://github.com/flatironinstitute/CaImAn.git
%cd '/content/CaImAn/'
!pip install -e .

# Install caiman dependencies (&> /dev/null will suppress the hundreds of printed lines in the output)
!pip install -r requirements.txt &> /dev/null

#import other dependencies
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio

#IMPORTANT! Newer versions of h5py will cause errors when saving results
!pip install h5py==2.10.0
import h5py

#Set up caiman
!python setup.py build_ext -i

#Other file setup
!python caimanmanager.py install --inplace

#Caiman imports
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.summary_images import local_correlations_movie_offline
from scipy.ndimage import center_of_mass
from IPython.display import display, clear_output

Cloning into 'CaImAn'...
remote: Enumerating objects: 24960, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 24960 (delta 0), reused 2 (delta 0), pack-reused 24954
Receiving objects: 100% (24960/24960), 518.36 MiB | 28.97 MiB/s, done.
Resolving deltas: 100% (16746/16746), done.
Checking out files: 100% (317/317), done.
/content/CaImAn
Obtaining file:///content/CaImAn
  Running setup.py develop for caiman
     |████████████████████████████████| 2.9 MB 22.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
running build_ext
Installed /root/caiman_data


In [3]:
#logging and configuring enviroment for interactive visualizations
#Some of this is redundant

try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    print(1)
except:
    print('NOT IPYTHON')

from ipyparallel import Client
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil
from scipy.ndimage.filters import gaussian_filter
import sys

import caiman as cm
from caiman.utils.visualization import nb_view_patches3d
import caiman.source_extraction.cnmf as cnmf
from caiman.components_evaluation import evaluate_components, estimate_components_quality_auto
from caiman.cluster import setup_cluster
from caiman.paths import caiman_datadir

import bokeh.plotting as bpl
bpl.output_notebook()

#uncomment this to enable detailed logging for debugging

# logging.basicConfig(format=
#                           "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
#                     # filename="/tmp/caiman.log",
#                     level=logging.DEBUG)

1


## Get paths to movie files and labelled ROI masks - this was used for our model tuning where we looped over these pairs, only here to collect these pairs to demonstate function below working

In [4]:
# #get a list of our masks and a list of our stacks in the same order
# from os import listdir
# maskpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs'
# mask_filenames = [f for f in listdir(maskpath) if 'manualROIs' in f]
# mask_filenames = sorted(mask_filenames)
# mask_filenames

In [5]:
# stack_indices = [x[:10] for x in mask_filenames] #get the index portion of the masks
# stack_celltypes = [x[22:(len(x)-4)] for x in mask_filenames] #get the cell type descriptions

# #create filenames in mat and h5, well only keep what exists
# stack_filenames_h5 = ['CLEAN_' + stack_indices[x] + '_stackRaw_mc_' + stack_celltypes[x] \
#                    + '_.h5' for x in range(len(mask_filenames))] #reconstruct filenames for the movies we have masks for
# stack_filenames_mat = ['CLEAN_' + stack_indices[x] + '_stackRaw_mc_' + stack_celltypes[x] \
#                    + '_.mat' for x in range(len(mask_filenames))] #reconstruct filenames for the movies we have masks for
# stack_filenames = stack_filenames_h5 + stack_filenames_mat 
# stack_filenames

In [6]:
# #checking that files exist in our cleaned file
# stackpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED'
# stackfiles = [f for f in listdir(stackpath) if f in stack_filenames]
# stackfiles = sorted(stackfiles)
# stackfiles

In [7]:
# #convert filenames back into paths
# maskpaths = [maskpath+'/'+f for f in mask_filenames]
# stackpaths = [stackpath+'/'+f for f in stackfiles]

In [8]:
# #join the lists into pairs of tuples
# mask_stack_pairs = list(map(lambda x, y:[x,y], maskpaths, stackpaths))
# mask_stack_pairs

## Initialize parameters object with starter values

In [9]:
#create parameters object
opts = params.CNMFParams()
#fname will be assigned in the loop
fnames = []
subfolder = 'stackRaw_mc'
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder

In [10]:
# set initial values for extraction and evaluation
# most of these are specific to our data and will not need to be changed during optimization

# overall params about our data

fr = 20                 # approximate frame rate of data - CONFIRMED FPS
decay_time = .4         # length of transient - CONFIRMED APPROPRIATE FOR OUR INDICATOR GCaMP6f
dims = [128, 256]       # dimensions of the FOV in pixels - CONFIRMED
dxy = [.29, .29]        # resolution of 1 pixel in um - CONFIRMED BY CARL

opts.set('data', {'fnames': fnames,
                   'fr': fr,
                   'decay_time': decay_time,
                   'dims': dims,
                   'dxy': dxy
                  })


# params related to the temporal traces

p = 0                   # order of the autoregressive system - 0 from carl's code
fudge_factor = 1        # (default is 0.96; Carl's value = 1) -- bias correction factor for discrete time constants
ITER = 2                # (default is 2; Carl's value=5) -- block coordinate descent iterations
tnb = 1                  # temporal global background components - TUNE

opts.set('temporal', {'p': p,
                      'fudge_factor': fudge_factor,
                      'ITER': ITER#,
                      #'nb': tnb
                 })

# p is also set in the preprocessing step
opts.set('preprocess', {'p': p
                 })



# params related to the FOV and patches for parallel processing

is_patches = False      # flag for processing in patches or not - turn on or off - Not used in Matlab

if is_patches:          # PROCESS IN PATCHES AND THEN COMBINE 
    rf = 25             # half size of each patch 
    stride = 5          # overlap between patches 
    K = 3               # number of components in each patch
    p_patch = p

else:                   # PROCESS THE WHOLE FOV AT ONCE
    rf = None           # setting these parameters to None
    stride = None       # will run CNMF on the whole FOV 
    K = 30              # number of neurons expected (in the whole FOV) - 40 from Carl's Code, seems to be too many

n_processes = 2         # Number of processes to run in parallel, 2 for 2 cores available in Colab

opts.set('patch', {'rf': rf,
                   'stride': stride,
                   'n_processes': n_processes,
                   "K": K
                  })   



# initialization params
ssub = 3               # spatial downsampling
tsub = 1                # temporal downsampling
ssub_B = 3              # background spatial downsampling
gSig = [5,5]            # radius (half-size) of average neurons (in pixels)
tau=0                   # standard deviation of neuron size along x and y - from Carl's code
nb = 1                  # number of background components
method_init = 'greedy_roi' #python Caiman defaults to greedy_roi, carl's code uses sparse_nmf, but sparse_nmf runs MUCH slower

opts.set('init', {'K': K,            # declared above in patch params    
                   'tau': tau,      
                   'tsub': tsub, 
                   'ssub': ssub, 
                   'ssub_B': ssub_B, 
                   'nb': nb,
                   'method_init': method_init
                  })

# parameters related to merging correlated ROIs
merge_thr = 0.95     # merging threshold, max correlation allowed - From Carl's Code

opts.set('merging', {'merge_thr': merge_thr
                            })


#set some spatial params
snb = 1                  # spatial global background components

opts.set('spatial', {'nb': snb
                            })

# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier (this will pick up only neurons
#           and filter out active processes)


#Not sure if these should be tuned or not

min_SNR = 2.5      # peak SNR for accepted components (if above this, acept)
SNR_lowest = 1         # minimum SNR for accepted components (if below this, reject)
rval_thr = 0.9     # space correlation threshold (if above this, accept)

use_cnn = True      # use the CNN classifier affects if 2 below params are used
min_cnn_thr = 0.9  # if cnn classifier predicts below this value, reject
cnn_lowest = 0.1   # neurons with cnn probability lower than this value are rejected

opts.set('quality', {'min_SNR': min_SNR,
                     'SNR_lowest': SNR_lowest,
                     'rval_thr': rval_thr,
                     'use_cnn': use_cnn,
                     'min_cnn_thr': min_cnn_thr,
                     'cnn_lowest': cnn_lowest})


#Manually assign subfolder variable
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder



#motion correction not used for our parameter tuning. Leaving these params here in case they get used in the future.

#%% First setup some parameters for data and motion correction
# dataset dependent parameters

# ADJUSTED FROM DEFAULTS TO CARL'S PARAMS ON 11/13 (not completely)

fr = 20             # imaging rate in frames per second
decay_time = 0.4    # length of a typical transient in seconds
dxy = (.29, .29)      # spatial resolution in x and y in (um per pixel)
max_shift_um = (12., 12.)       # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um

# motion correction parameters
pw_rigid = False       # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
# overlap between pathes (size of patch in pixels: strides+overlaps)
overlaps = (24, 24)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3

opts.set('motion', {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy'
})

## Seed CNMF with the masks

In [11]:
#Function to run seeded cnmf using masks, then return results

def seeded_cnmf(path_to_stack, path_to_masks, opts):
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)

  fnames = [path_to_stack]
  opts.set('data', {'fnames': fnames})

  try:

    #cluster handling
    if 'dview' in locals():
      cm.stop_server(dview=dview)
    dview = cm.cluster.start_server(ncpus=2) #Start a cluster with 2 CPU's (available in colab)


    #Read in masks and reformat
    g = h5py.File(path_to_masks, 'r')

    #transpose the matrix and save to an array A
    mask_A = g['bwMaskStack'][:].T

    #close the .mat file holding the masks
    g.close()

    #rearrange the dimensions of masks to match reformatted .h5 movie that has been flipped (not necessary if input movie has not been flipped), 
                                                                                                                  #but will afect following lines
    mask_A = mask_A.transpose(1,0,2)

    #reshape to 2D, first dimension is 128*256 (32768), 2nd dimension is the # of ROI's
    mask_A = mask_A.reshape((mask_A.shape[1]*mask_A.shape[0]), mask_A.shape[2])

    #convert the values from 0/1 to boolean False/True
    mask_A = np.array(mask_A, dtype=bool)
    print('mask read in and reformatted')

    #For seeded CNMF, need to verify certain params
    opts.patch['only_init'] = False
    opts.data['use_cnn'] = False

    print('params adjusted for seeded cnmf')
    
    #Initialize a new cnmf object and pass in our masks as the "Ain" param
    #"Ain" is A-in, meaning the A matrix holding the spatial footprints of the roi's
    cnm_seeded = cnmf.CNMF(n_processes = 2, params=opts, dview=dview, Ain=mask_A)
    print('seeded cnmf object initialized')
    cnm_seeded.fit_file(motion_correct = False, include_eval=True)
    print('seeded cnmf completed')

    cm.stop_server(dview=dview)


#Comment out whichever you don't want to return and leave the one you do

    #return either the whole fit CNMF object(with associated visualizations available)
    #return cnm_seeded

    #Or just return the spatial and temporal response (A and C matrices) in a dictionary, to be saved and used in our model
    return {'stack_name': path_to_stack[:(len(path_to_stack)-4)].split('/')[6], #chops the movie identifier out of the filepath
            'spatial': cnm_seeded.estimates.A,
            'temporal': cnm_seeded.estimates.C}

  except:
    print('failed')
    cm.stop_server(dview=dview)

In [12]:
#manually setting the paths to the example files in the new folder
#later we can pass these in with a wrapper function to extract everything in the folders

path_to_stack = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded CNMF/Preformatted Movies/EXAMPLE_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5'
path_to_masks = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded CNMF/Seed Masks/EXAMPLE_210728_0_1_manualROIs_tm2_tm9_syt.mat'

In [13]:
#Call the function, passing in a string path to the movie, path to masks, and the params object('opts')

seeded = seeded_cnmf(path_to_stack = path_to_stack, #from the pairs of files above, could be swapped out for string directly
                     path_to_masks = path_to_masks, #same as above
                     opts = opts) #params object globally initialized earlier

#uncomment this line if returning the full object to visualize
#seeded.estimates.nb_view_components(denoised_color = 'red')

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
...............mask read in and reformatted
params adjusted for seeded cnmf
seeded cnmf object initialized
spatial support for each components given by the user
USING MODEL:/root/caiman_data/model/cnn_model.json
1/1 [==============================] - 0s 477ms/step
....seeded cnmf completed


## test output dictionary and save - this will all not run if the function output above is edited to output the cnmf object for visualization

In [14]:
#check the stack name saved in the dictionary
print(seeded['stack_name'])

EXAMPLE_210728_0_1_stackRaw_mc_tm2_tm9_syt


In [15]:
#temporal responses are saved in the dictionary under 'temporal'
#showing shape here just for demonstration
seeded['temporal'].shape

(12, 5519)

In [16]:
#spatial footprints are saved in the dictionary under 'spatial'
seeded['spatial'].shape

(32768, 12)

In [17]:
#procedurally generate the path to the result file we will write
results_folder_path ='/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded CNMF/Results/'
result_h5_path = results_folder_path + seeded['stack_name'] + '_result.h5'
result_h5_path

'/content/drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded CNMF/Results/EXAMPLE_210728_0_1_stackRaw_mc_tm2_tm9_syt_result.h5'

In [18]:
#Write the results into an hdf5 file
with h5py.File(result_h5_path, "w") as result_file:
  result_file.create_dataset('stack_name', data=seeded['stack_name'])
  result_file.create_dataset('temporal', data=seeded['temporal'])
  result_file.create_dataset('spatial', data=seeded['spatial'].A) # ".A" converts from a sparse matrix to normal. h5py cant save sparse matrices


In [19]:
#Read those results back to test
results_read = h5py.File(result_h5_path,'r')
results_read.keys()

<KeysViewHDF5 ['spatial', 'stack_name', 'temporal']>

In [20]:
#verify that everything wrote to the file is identical to the original results
print(results_read['spatial'][:].all() == seeded['spatial'].A.all())
print(results_read['temporal'][:].all() == seeded['temporal'].all())
print(results_read['stack_name'][()] == seeded['stack_name'])

True
True
True


In [21]:
#close the results file
results_read.close()